In [1]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np

from c3.libraries import fidelities
from c3.parametermap import ParameterMap
from utils import *
from c3.experiment import Experiment
import c3.libraries.constants as constants
import c3.utils.qt_utils as qt_utils

2021-08-18 16:37:03.820494: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-18 16:37:03.820528: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# plotting functions
def plotOccupations(
        experiment: Experiment,
        populations: np.array,
        gate_sequence: List[str],
        level_names: List[str] = None,
        filename: str = None,
) -> None:
    """
    Plots time dependent populations. They need to be calculated with `runTimeEvolution` first.

    Parameters
    ----------
    experiment: Experiment
        The experiment containing the model and propagators
    populations: np.array
        Population vector for each time step
    gate_sequence: List[str]
        List of gate names that will be applied to the state
    level_names: List[str]
        Optional list of names for the levels. If none, the default list
        from the experiment will be used.
    filename: str
        Optional name of the file to which the plot will be saved. If none,
        it will only be shown.

    Returns
    -------

    """
    # plot populations
    fig, axs = plt.subplots(1, 1)
    dt = experiment.ts[1] - experiment.ts[0]
    ts = np.linspace(0.0, dt * populations.shape[1], populations.shape[1])
    axs.plot(ts / 1e-9, populations.T)

    # plot vertical lines
    gate_steps = [experiment.partial_propagators[g].shape[0] for g in gate_sequence]
    for i in range(1, len(gate_steps)):
        gate_steps[i] += gate_steps[i - 1]
    gate_times = gate_steps * dt
    plt.vlines(gate_times / 1e-9, tf.reduce_min(populations), tf.reduce_max(populations),
               linestyles=':', colors="black")

    # set plot properties
    axs.tick_params(direction="in", left=True, right=True, top=False, bottom=True)
    axs.set_xlabel('Time [ns]')
    axs.set_ylabel('Population')
    plt.legend(level_names if level_names else experiment.pmap.model.state_labels)
    plt.tight_layout()

    # show and save
    if filename:
        print("saving plot in " + filename)
        plt.savefig(filename)
    else:
        plt.show()
    plt.close()


def plotSignal(time, signal, filename=None, spectrum_cut=1e-4) -> None:
    """
    Plots a time dependent signal and its normalised frequency spectrum.

    Parameters
    ----------
    time
        timestamps
    signal
        signal value
    filename: str
        Optional name of the file to which the plot will be saved. If none,
        it will only be shown.
    spectrum_cut:
        If not None, only the part of the normalised spectrum will be plotted
        whose absolute square is larger than this value.

    Returns
    -------

    """
    # plot time domain
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    axs[0].set_title('Signal')
    axs[0].plot(time, signal)
    axs[0].set_xlabel('time')

    # calculate frequency spectrum
    n_samples = time.shape[-1]
    freq_signal = np.fft.rfft(signal)
    normalised = freq_signal / np.max(freq_signal)
    freq = np.fft.rfftfreq(n_samples, time[-1] / n_samples)
    print("fft: ", n_samples, len(freq), len(normalised))

    # cut spectrum if necessary
    if spectrum_cut is not None:
        limits = np.flatnonzero(np.abs(normalised) ** 2 > 1e-4)
        freq = freq[limits[0]:limits[-1]]
        normalised = normalised[limits[0]:limits[-1]]

    # plot frequency domain
    axs[1].set_title('Spectrum')
    axs[1].plot(freq, normalised.real, label="Re")
    axs[1].plot(freq, normalised.imag, label="Im")
    axs[1].plot(freq, np.abs(normalised) ** 2, label="Square")
    axs[1].set_xlabel('frequency')
    axs[1].legend()

    # show and save
    plt.tight_layout()
    if filename:
        print("saving plot in " + filename)
        plt.savefig(filename)
    else:
        plt.show()
    plt.close()

In [3]:
def createPulse(t_final: float, sigma_factor: float) -> None:
    # preparation
    occupied_levels = [0, 2]
    directory = "./output"
    output_dir = pathlib.Path(directory)
    output_dir.mkdir(parents=True, exist_ok=True)

    # model
    q1 = createQubit(1, 5, 5e9, -300e6)
    model = createModel([q1])
    generator = createGenerator(model)

    # gate
    envelope = createPWCGaussianPulse(t_final, t_final / sigma_factor, 30)
    ts = np.linspace(0, t_final, 10000)
    plotSignal(ts, envelope.get_shape_values(ts), directory + f"/envelope_{sigma_factor}.png")
    '''
    ideal = qt_utils.np_kron_n([
        constants.Id,
        constants.x90p,
    ])
    #print('ideal gate:\n', np.round(ideal, 3))
    gate = createSingleQubitGate("lower-X", t_final, 5e9, envelope, model, q1, ideal)
    gates = [gate]
    gate_names = list(map(lambda g: g.get_key(), gates))
    '''

    # experiment
    '''
    exp = Experiment(pmap=ParameterMap(instructions=gates, model=model, generator=generator))
    exp.set_opt_gates(gate_names)
    exp.compute_propagators()
    '''

    # initial state
    '''
    init_state = createState(model, occupied_levels)
    state = init_state.numpy().flatten()
    print("initial state=", state, ", occupation=", exp.populations(state, model.lindbladian).numpy())
    '''

    # time evolution and signal before optimisation
    '''
    sequence = ["lower-X[0]"]
    populations = runTimeEvolutionDefault(exp, init_state, sequence)
    plotOccupations(exp, populations, sequence, filename=directory + f"/populations_{sigma_factor}.png")
    signal = generator.generate_signals(gate)[getDrive(model, q1).name]
    plotSignal(signal['ts'], signal['values'], directory + f"/signal_{sigma_factor}.png", spectrum_cut=1e-4)
    '''


def runExperiment(envelope: pulse.Envelope, file_suffix: str) -> None:
    # preparation
    active_levels = 4
    occupied_levels = [0, 2]
    directory = "./output"
    output_dir = pathlib.Path(directory)
    output_dir.mkdir(parents=True, exist_ok=True)

    # model
    q1 = createQubit(1, 5, 5e9, -300e6)
    model = createModel([q1])
    generator = createGenerator(model)

    # gate
    t_final = 7e-9
    #envelope = createPWCGaussianPulse(t_final, t_final / 4, 30)
    ideal = qt_utils.np_kron_n([
        constants.Id,
        constants.x90p,
    ])
    print('ideal gate:\n', np.round(ideal, 3))
    gate = createSingleQubitGate("lower-X", t_final, 5e9, envelope, model, q1, ideal)
    gates = [gate]
    gate_names = list(map(lambda g: g.get_key(), gates))

    # experiment
    exp = Experiment(pmap=ParameterMap(instructions=gates, model=model, generator=generator))
    exp.set_opt_gates(gate_names)
    unitaries = exp.compute_propagators()
    print('unitaries: ', dict(map(lambda kv: (kv[0], kv[1].numpy().shape), unitaries.items())))
    print(unitaries)

    # initial state
    init_state = createState(model, occupied_levels)
    state = init_state.numpy().flatten()
    print("initial state=", state, ", occupation=", exp.populations(state, model.lindbladian).numpy())

    # time evolution and signal before optimisation
    sequence = ["lower-X[0]"]
    populations = runTimeEvolutionDefault(exp, init_state, sequence)
    plotOccupations(exp, populations, sequence, filename=directory + f"/populations_before_{file_suffix}.png")
    signal = generator.generate_signals(gate)[getDrive(model, q1).name]
    plotSignal(signal['ts'], signal['values'], directory + f"/signal_before_{file_suffix}.png", spectrum_cut=1e-4)
    ts = tf.convert_to_tensor(np.linspace(0, t_final, 1000))
    shape = envelope.get_shape_values(ts)
    print(shape)

    # add all optimisable parameters to a map
    drives = filterValues(model.couplings, chip.Drive)
    gateset_opt_map = []
    for gate in gates:
        for target in gate.targets:
            # TODO: target as index might not always work
            drive = drives[target]
            gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "amp")])
            gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "freq_offset")])
            gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "xy_angle")])
            gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "delta")])
            gateset_opt_map.append([(gate.get_key(), drive.name, "carrier", "framechange")])

    # optimise
    optimisable_gates = list(filter(lambda g: g.get_key() != "id[]", gates))
    callback = lambda fidelity: print(fidelity)
    params_before, final_fidelity, params_after = optimise(
        exp, optimisable_gates,
        optimisable_parameters=gateset_opt_map,
        fidelity_fctn=fidelities.state_transfer_infid_set,
        fidelity_params={
            'psi_0': init_state[:active_levels],
            'active_levels': 4,
        },
        callback=callback,
        log_dir=(directory + ("/log_{0}_{1:.2f}/".format(file_suffix, t_final * 1e9)))
    )
    print('before:\n', params_before)
    print('after:\n', params_after)
    print('fidelity:\n', final_fidelity)

    # time evolution and signal after optimisation
    populations = runTimeEvolutionDefault(exp, init_state, sequence)
    plotOccupations(exp, populations, sequence, filename=directory + f"/populations_after_{file_suffix}.png")
    signal = generator.generate_signals(gate)[getDrive(model, q1).name]
    plotSignal(signal['ts'], signal['values'], directory + f"/signal_after_{file_suffix}.png", spectrum_cut=1e-4)

'''
directory = "./output"
output_dir = pathlib.Path(directory)
output_dir.mkdir(parents=True, exist_ok=True)

t_final = 7e-9
sigma_factors = np.arange(4, 200, 5)
for sigma_factor in sigma_factors:
    print(sigma_factor, t_final / sigma_factor)
    envelope = createPWCGaussianPulse(t_final, t_final / sigma_factor, 40)
    ts = np.linspace(0, t_final, 10000)
    vals = envelope.get_shape_values(ts)
    print(vals)
    plotSignal(ts, vals, directory + f"/envelope_{sigma_factor}.png")
'''

t_final = 7e-9
runExperiment(createGaussianPulse(t_final, t_final / 4), 'gaussian')
print('======================================================')
runExperiment(createPWCGaussianPulse(t_final, t_final / 4, 30), 'pwcgauss')
print('======================================================')
runExperiment(createPWCConstantPulse(t_final, 30), 'pwc')

2021-08-18 16:37:08.533061: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-18 16:37:08.533350: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-18 16:37:08.533370: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-18 16:37:08.533392: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist
2021-08-18 16:37:08.533980: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


ideal gate:
 [[0.707+0.j    0.   -0.707j 0.   +0.j    0.   -0.j   ]
 [0.   -0.707j 0.707+0.j    0.   -0.j    0.   +0.j   ]
 [0.   +0.j    0.   -0.j    0.707+0.j    0.   -0.707j]
 [0.   -0.j    0.   +0.j    0.   -0.707j 0.707+0.j   ]]


2021-08-18 16:37:09.515757: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-18 16:37:09.532500: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2793690000 Hz


unitaries:  {'lower-X[0]': (5, 5)}
{'lower-X[0]': <tf.Tensor: shape=(5, 5), dtype=complex128, numpy=
array([[ 5.03419798e-01-8.38151305e-02j, -1.00215525e-01-8.53932437e-01j,
         1.13611063e-03-1.72759459e-02j,  6.12976857e-05-1.42560418e-04j,
        -1.12337675e-05-2.18188977e-06j],
       [-6.91018188e-02-8.56932898e-01j,  5.10264616e-01-1.62566022e-02j,
        -9.08033703e-03-1.31971606e-02j, -2.49891979e-04-1.08865983e-04j,
         9.86767729e-06-1.65215837e-05j],
       [-1.07523500e-03-2.08151393e-02j, -5.29407808e-03-9.69093208e-03j,
         7.75635007e-01+6.30475711e-01j,  1.79739995e-02-3.35196994e-03j,
         1.69906764e-04-1.33006380e-04j],
       [-3.20419971e-05-2.27032232e-04j, -2.11493170e-04+3.33860813e-06j,
         1.70488796e-02-6.60713363e-03j, -3.28020973e-01+9.44267218e-01j,
        -9.87570404e-04+2.06463416e-02j],
       [ 1.14039254e-05-3.51771545e-06j, -6.86682321e-06+2.04869356e-05j,
         1.54444760e-05-1.70681464e-04j,  1.05051508e-02+1.780184